## Chicago Food Inspections
#### CGDL Group

In the modern world, consumers are more empowered than ever. They increasingly use freely- available tools on the internet to make informed decisions while relying less upon traditional trust-building sources such as word-of-mouth.While internet sources such as Yelp can provide valuable information, there remains significant opportunity to enhance the quality of data readily available to consumers.

To further empower consumers in the City of Chicago, CDGL Group plans to extract, analyze and data collected by the Chicago Department of Public Health Food Protection Program. These results will underpin a set of tools to improve customer access to the following information:

- Restaurant and other food establishment recommendations 
- Cleanliness score
- Locations with clean restaurants

In [1]:
import pandas as pd # table manipulation
import numpy as np # number manipulation

from datetime import datetime,timedelta # time metrics
script_start = datetime.now() # Start Script

import re # string manipulation
import usaddress as add # address parsing

import configparser # read in MySQL connection attributes stored in a separate config file
import pymysql # import mysql package to fetch data in Python
from sqlalchemy import create_engine # SQL Package that helps connect to MySQL for DB Writes

# Add encoder of np.float64
pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
pymysql.converters.encoders[np.int64] = pymysql.converters.escape_int
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

In [2]:
# Create function to input Chicago Food Inspections dataset
def inputData(limit=50000,offset=0,start_date="",condition=">="):
    # Initiate df
    columns = ['inspection_id', 'dba_name', 'aka_name', 'license_', 'facility_type',
           'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
           'inspection_type', 'results', 'violations', 'latitude', 'longitude',
           'location']
    df = pd.DataFrame(columns=columns)
    
    # Input variables
    limit = limit # Able to pull 50k at a time
    offset = offset
    counter = 1
    rows = limit # Only to start while loop

    # Need to subset data by Inspection Date?
    if start_date != "":
        start_filter = "&$where=inspection_date"+condition+"'"+start_date+"'"
    else:
        start_filter = start_date

    start = datetime.now()

    # Conduct while loop to paginate through dataset until all data for query retrieved
    while limit == rows:
        iter_start = datetime.now()
        df_temp = pd.read_json("https://data.cityofchicago.org/resource/4ijn-s7e5.json?$limit="+
                          str(limit)+"&$offset="+str(offset)+"&$order=inspection_date"+start_filter)
        print("Time Between Iteration",counter,"-",datetime.now()-iter_start,"-",datetime.now()-start)
        rows = df_temp.shape[0]
        offset += limit
        df = pd.concat([df,df_temp])
        counter += 1
    print("Finished")
    
    return df

In [3]:
# Insert data for last 7 days
week_ago = (datetime.now() - timedelta(days=7)).date()
#df = inputData(start_date=week_ago,condition=">=").reset_index()

# Insert full dataset
df = inputData().reset_index()
df.head()

Time Between Iteration 1 - 0:01:59.600748 - 0:01:59.600765
Time Between Iteration 2 - 0:00:10.977990 - 0:02:10.648567
Time Between Iteration 3 - 0:00:20.639913 - 0:02:31.489834
Time Between Iteration 4 - 0:00:39.301809 - 0:03:11.118168
Time Between Iteration 5 - 0:00:30.956221 - 0:03:42.550447
Finished


,index,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,0,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,2010-01-04T00:00:00.000,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"{'latitude': '-87.6447545707008', 'longitude':..."
1,1,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,2010-01-04T00:00:00.000,License,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,NaN,NaN,NaN
2,2,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,2010-01-04T00:00:00.000,Tag Removal,Pass,NaN,41.884586,-87.631010,"{'latitude': '-87.63101044588599', 'longitude'..."
3,3,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,2010-01-04T00:00:00.000,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"{'latitude': '-87.6280091630558', 'longitude':..."
4,4,67732,WOLCOTT'S,TROQUET,1992039.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,2010-01-04T00:00:00.000,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"{'latitude': '-87.67596676683779', 'longitude'..."


Create functions `write_query` and `read_query` to insert and read data from that insert as a sanity check

In [4]:
# Create function to insert data into MySQL relational instance
def write_query(sql,records,schema='foodinspection',execute='Many'):
    
    try:
    # Pull config information: Connection, Username, Password
        config = configparser.ConfigParser()
        config.read('food-inspection-config.ini')
        conn = config.get('mysql','Connection')
        user = config.get('mysql','Username')
        pwd = config.get('mysql','Password')

        # Connect to Sakila
        db = pymysql.connect(conn,user,pwd,schema)
        
        # Write SQL Query
        cursor = db.cursor()
        
        # Insert from Python or SQL
        if execute == 'Many':
            cursor.executemany(sql,records)
        else:
            cursor.execute(sql)
            
        db.commit()
        print("Successful Insert!")
    except Exception as e:
        print(e)
    finally:
        db.close()

In [5]:
# Create function to read data into MySQL relational instance
def read_query(sql,schema='foodinspection',head=False):
    
    try:
        # Pull config information: Connection, Username, Password
        config = configparser.ConfigParser()
        config.read('food-inspection-config.ini')
        conn = config.get('mysql','Connection')
        user = config.get('mysql','Username')
        pwd = config.get('mysql','Password')

        # Connect to Sakila
        db = pymysql.connect(conn,user,pwd,schema)

        # Execute dataframe and output either first 5 rows or all depending on user choice
        if head == True:
            df = pd.read_sql(sql, con = db).head()
        else:
            df = pd.read_sql(sql, con = db)

        return df
    except Exception as e:
        print(e)

### Generate DML scripts for the `foodinspection` OLTP schema

Prepare `risk` table insert

In [6]:
# Create risk list
risks = list(set(df.risk))
risks = [x for x in risks if str(x) != 'nan']

# Create risk_id list
nums = []
for risk in risks:
    if "1" in risk:
        nums.append(1)
    elif "2" in risk:
        nums.append(2)
    elif "3" in risk:
        nums.append(3)
    else:
        nums.append(4)
    
# Create tuple of rows to insert
records = [(nums[i],risks[i]) for i in range(0,len(risks))]

# Insert data
sql = """INSERT IGNORE INTO risk (risk_id,risk)
        VALUES (%s,%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM risk LIMIT 5")

Successful Insert!


,risk_id,risk
0,1,Risk 1 (High)
1,2,Risk 2 (Medium)
2,3,Risk 3 (Low)
3,4,All


Prepare `result` table insert

In [7]:
results_orig = list(set(df.results))

# Create result and conditon_flag columnns
results = []
conditions = []
for i in results_orig:
    if "w/ Condititions" in i:
        conditions.append(True)
    else:
        conditions.append(False)
    
    if "Pass" in i:
        results.append("Pass")
    else:
        results.append(i)
        
# Create tuple of rows to insert
records = list(set([(results[i],conditions[i]) for i in range(0,len(results))]))

# Insert data
sql = """INSERT IGNORE INTO result (result,condition_flag)
        VALUES (%s,%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM result LIMIT 5")

Successful Insert!


,result_id,result,condition_flag
0,110,Not Ready,0
1,111,Business Not Located,0
2,112,Fail,0
3,113,Out of Business,0
4,114,No Entry,0


Prepare `violation` table insert

In [8]:
# Initiate violation df
viodf = df[["inspection_id","violations"]]
viodf = viodf[viodf.violations.notnull()]

# Split multiple violations frome each inspection id to rows
new_viodf = pd.DataFrame(viodf.violations.str.split("|").tolist(),index=viodf.inspection_id).stack()
new_viodf = new_viodf.reset_index([0,'inspection_id'])
new_viodf.columns = ['inspection_id','violation']

# Get rid of initial violation id
violations = new_viodf.violation.str.split('. ',n=1,expand=True)
violations.columns = ['number','violation']

# Split comments from violation
comments = violations.violation.str.split('- Comments:',n=1,expand=True)
comments.columns = ['violation','comment']

# Create inspection violation df
ins_vio_df = pd.DataFrame({'inspection_id':new_viodf.inspection_id,
                           'violation':comments.violation,
                           'comment':comments.comment})

# Create unique set of violations
records = list(set(ins_vio_df.violation))
records = [tuple([i]) for i in records]

# Insert data
sql = """INSERT IGNORE INTO violation (violation)
        VALUES (%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM violation LIMIT 5")

Successful Insert!


,violation_id,violation
0,1,NO SMOKING REGULATIONS
1,2,FOOD-CONTACT SURFACES: CLEANED & SANITIZED
2,3,"ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, PROPE..."
3,4,SEWAGE & WASTE WATER PROPERLY DISPOSED
4,5,PREVIOUS CORE VIOLATION CORRECTED


Prepare `establishment` table insert

In [9]:
# Reverse tuple of address qualifiers to place in dictionary
def reverseTuple(x):
    return (x[1],x[0])

# Extract address and create list of address components
def extractAddress(full_address):
    
    # Initialize lists
    try:
        add_temp = (add.parse(full_address))
        add_dict = dict(map(reverseTuple,add_temp))

        # Address number list
        if "AddressNumber" in add_dict:
            add_num = add_dict["AddressNumber"]
        else:
            add_num = ""

        # Address direction list
        if "StreetNamePreDirectional" in add_dict:
            add_dir = add_dict["StreetNamePreDirectional"]
        else:
            add_dir = ""

        # Address street list
        if "StreetName" in add_dict and "StreetNamePostType" in add_dict and "OccupancyIdentifier" in add_dict:
            add_street = add_dict["StreetName"] + " " + add_dict["StreetNamePostType"] + " " + add_dict["OccupancyIdentifier"]
        elif "StreetName" in add_dict and "StreetNamePostType" in add_dict:
            add_street = add_dict["StreetName"] + " " + add_dict["StreetNamePostType"]
        elif "StreetName" in add_dict:
            add_street = add_dict["StreetName"]
        else:
            add_street = ""

        # Address city list
        if "PlaceName" in add_dict:
            add_city = add_dict["PlaceName"]
        else:
            add_city = ""

        # Address state list
        if "StateName" in add_dict:
            add_state = add_dict["StateName"]
        else:
            add_state = ""

        # Address zip list
        if "ZipCode" in add_dict:
            add_zip = add_dict["ZipCode"]
        else:
            add_zip = ""
        
    # Error handling in the case that address is blank
    except:
        add_num = ""
        add_dir = ""
        add_street = ""
        add_city = ""
        add_state = ""
        add_zip = ""
    
    # Replace NaNs as they cannot go into MySQL database
    add_num = add_num.replace('nan','')
    add_dir = add_dir.replace('nan','')
    add_street = add_street.replace('nan','')
    add_city = add_city.replace('nan','')
    add_state = add_state.replace('nan','')
    add_zip = add_zip.replace('nan','')
        
    return add_num, add_dir, add_street, add_city, add_state, add_zip

In [10]:
# Clean address
df["address"] = df.address.replace(" - ","-").replace(" -","-").replace("- ","-").fillna("")

# Clean city
df["city"] = df.city.fillna("")

# Clean zip code
df["zip"] = df.zip.apply(str)
df["zip"] = df.zip.fillna("")

# Clean state
df["state"] = df.state.fillna("")

# Clean facility type
df["facility_type"] = df.facility_type.fillna("")

# Create full address field for extractAddress function
full_address = df.address+df.city+" "+df.state+" "+df.zip

# Clean dba_name
df["dba_name"] = df.dba_name.fillna("")

# Clean aka_name
df["aka_name"] = df.aka_name.fillna("")

# Clean Latitude and Longitude
latitudes = [i if abs(i) > 0 else None for i in df.latitude]
longitudes = [i if abs(i) > 0 else None for i in df.longitude]

# Parse addresses
address_list = list(map(extractAddress,full_address.fillna("")))

# Create establishment table records with potential duplicates
records = list(set([(df.dba_name[i],df.aka_name[i])+address_list[i]+
                     (latitudes[i],longitudes[i],df.facility_type[i]) for i in range(0,df.shape[0])]))

# Insert data
sql = """INSERT IGNORE INTO establishment (dba_name,aka_name,address_num,address_direction,street,city,state,
                                            zip,latitude,longitude,facility_type)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM establishment LIMIT 5")

Successful Insert!


,establishment_id,dba_name,aka_name,address_num,address_direction,street,city,state,zip,latitude,longitude,facility_type
0,110,SUBWAY,SUBWAY,6450,S,PULASKI RD,CHICAGO,IL,60629.0,41.775287,-87.722819,
1,111,CELIS PIZZERIA INC,CELIS PIZZERIA INC,2605,S,KOSTNER AVE,CHICAGO,IL,60623.0,41.843910,-87.734130,Restaurant
2,112,VINNIES SUB SHOP,VINNIES SUB SHOP,1204,W,GRAND AVE,CHICAGO,IL,60642.0,41.891187,-87.657667,Restaurant
3,113,RED MANGO,RED MANGO,151,E,Randolph ST,CHICAGO,IL,60601.0,41.884167,-87.622603,Restaurant
4,114,OWEN,OWEN,8247,S,Christiana,CHICAGO,IL,60652.0,41.742872,-87.705713,School


Prepare `inspection` table insert

In [11]:
# Convert date string to date format YYYY-mm-dd
def datetrans(x):
    return x[:10]

In [12]:
# Pull tables to pull the Ids
establishments = read_query("""SELECT * FROM establishment""")
risks = read_query("""SELECT * FROM risk""")
results = read_query("""SELECT * FROM result""")
results["results"] = [result[i]+" w/ Conditions" if results.condition_flag[i] else results.result[i] for i in range(0,len(results))]

# Create dataframe with merged data
df_link = df.merge(establishments, on=["dba_name","aka_name","latitude","longitude"],how="left")
df_link = df_link.merge(risks,on=["risk"],how="left")
df_link = df_link.merge(results,on=["results"],how="left")
df_link = df_link.where(pd.notnull(df_link), None)

# Find inspection table columns
inspection = df_link[['inspection_id','establishment_id','risk_id','inspection_date','inspection_type','result_id']].reset_index()
inspection["inspection_date"] = list(map(datetrans,inspection.inspection_date))
records = list(inspection.drop(columns=['index']).to_records())
records = [tuple(list(i)[1:]) for i in records]

# Insert data
sql = """INSERT IGNORE INTO inspection (inspection_id,establishment_id,risk_id,
            inspection_date,inspection_type,result_id)
        VALUES (%s,%s,%s,%s,%s,%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM inspection LIMIT 5")

Successful Insert!


,inspection_id,establishment_id,risk_id,inspection_date,inspection_type,result_id
0,44247,17841,1,2010-01-05,Complaint,115
1,44248,13468,1,2010-01-21,Canvass,115
2,44249,7561,1,2010-01-21,Canvass Re-Inspection,115
3,44250,25921,1,2010-02-09,Canvass,115
4,44251,25921,1,2010-02-09,Canvass,115


Prepare `inspection_violation` table insert

In [13]:
# Read violation table to obtain ids
violation_df = read_query("select * from violation")

# Join on violation from previous dataframe
inspection_violation = ins_vio_df.merge(violation_df,on='violation')[['inspection_id','violation_id','comment']]
records = list(inspection_violation.to_records())
records = [tuple(list(i)[1:]) for i in records]

# Insert data
sql = """INSERT IGNORE INTO inspection_violation (inspection_id,violation_id,comment)
        VALUES (%s,%s,%s)"""
write_query(sql, records)

# Read test
read_query("SELECT * FROM inspection_violation LIMIT 5")

Successful Insert!


,inspection_id,violation_id,comment
0,67738,31,must provide pest control log book.
1,104236,31,All necessary control measures shall be used ...
2,118297,31,All necessary control measures shall be used ...
3,67741,31,All necessary control measures shall be used ...
4,80207,31,All necessary control measures shall be used ...


### Generate DML scripts for the `foodinspectionDW` OLAP schema

Prepare `dim_risk` table insert

In [14]:
# Insert data
sql = """INSERT IGNORE INTO dim_risk (risk_id,risk)
        SELECT risk_id, risk from foodinspection.risk"""
write_query(sql, records,schema='foodinspectionDW',execute='One')

# Read test
read_query("SELECT * FROM dim_risk LIMIT 5",schema='foodinspectionDW')

Successful Insert!


,risk_id,risk
0,1,Risk 1 (High)
1,2,Risk 2 (Medium)
2,3,Risk 3 (Low)
3,4,All


Prepare `dim_result` table insert

In [15]:
# Insert data
sql = """INSERT IGNORE INTO dim_result (result_id,result,condition_flag)
        SELECT result_id, result, condition_flag from foodinspection.result"""
write_query(sql, records,schema='foodinspectionDW',execute='One')

# Read test
read_query("SELECT * FROM dim_result LIMIT 5",schema='foodinspectionDW')

Successful Insert!


,result_id,result,condition_flag
0,110,Not Ready,0
1,111,Business Not Located,0
2,112,Fail,0
3,113,Out of Business,0
4,114,No Entry,0


Prepare `dim_establishment` table insert

In [16]:
# Insert data
sql = """INSERT IGNORE INTO dim_establishment (establishment_id, dba_name, aka_name, address_num,
       address_direction, street, city, state, zip, latitude, longitude, facility_type)
        SELECT establishment_id, dba_name, aka_name, address_num,
       address_direction, street, city, state, zip, latitude, longitude, facility_type 
       from foodinspection.establishment"""
write_query(sql, records,schema='foodinspectionDW',execute='One')

# Read test
read_query("SELECT * FROM dim_establishment LIMIT 5",schema='foodinspectionDW')

Successful Insert!


,establishment_id,dba_name,aka_name,address_num,address_direction,street,city,state,zip,latitude,longitude,facility_type
0,110,SUBWAY,SUBWAY,6450,S,PULASKI RD,CHICAGO,IL,60629.0,41.775287,-87.722819,
1,111,CELIS PIZZERIA INC,CELIS PIZZERIA INC,2605,S,KOSTNER AVE,CHICAGO,IL,60623.0,41.843910,-87.734130,Restaurant
2,112,VINNIES SUB SHOP,VINNIES SUB SHOP,1204,W,GRAND AVE,CHICAGO,IL,60642.0,41.891187,-87.657667,Restaurant
3,113,RED MANGO,RED MANGO,151,E,Randolph ST,CHICAGO,IL,60601.0,41.884167,-87.622603,Restaurant
4,114,OWEN,OWEN,8247,S,Christiana,CHICAGO,IL,60652.0,41.742872,-87.705713,School


Prepare `fact_inspection` table insert

In [17]:
# Insert data
sql = """INSERT IGNORE INTO fact_inspection (inspection_id,establishment_id,inspection_date,
    inspection_type,violation,comment,risk_id,high_risk,medium_risk,low_risk,all_risk,result_id,pass,fail)
    SELECT i.inspection_id,e.establishment_id,i.inspection_date,i.inspection_type,
        v.violation,iv.comment,ri.risk_id,
        CASE WHEN ri.risk like '%High%' THEN 1 ELSE 0 end as high_risk,
        CASE WHEN ri.risk like '%Medium%' THEN 1 ELSE 0 end as medium_risk,
        CASE WHEN ri.risk like '%Low%' THEN 1 ELSE 0 end as low_risk,
        CASE WHEN ri.risk like '%All%' THEN 1 ELSE 0 end as all_risk,
        re.result_id, 
        CASE WHEN re.result like '%Pass%' THEN 1 ELSE 0 end as pass,
        CASE WHEN re.result like '%Fail%' THEN 1 ELSE 0 end as fail
    FROM foodinspection.inspection i
            LEFT JOIN foodinspection.establishment e on i.establishment_id = e.establishment_id
            LEFT JOIN foodinspection.risk ri on i.risk_id = ri.risk_id
            LEFT JOIN foodinspection.result re on i.result_id = re.result_id
            LEFT JOIN foodinspection.inspection_violation iv on i.inspection_id = iv.inspection_id
            INNER JOIN foodinspection.violation v on iv.violation_id = v.violation_id
            """
write_query(sql, records,schema='foodinspectionDW',execute='One')

# Read test
read_query("SELECT * FROM fact_inspection LIMIT 5",schema='foodinspectionDW')

Successful Insert!


,inspection_violation_id,inspection_id,establishment_id,inspection_date,inspection_type,violation,comment,risk_id,high_risk,medium_risk,low_risk,all_risk,result_id,pass,fail
0,110,67738,27696,2010-01-04,License,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,must provide pest control log book.,1,1,0,0,0,112,0,1
1,111,104236,34141,2010-01-04,Canvass,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,All necessary control measures shall be used ...,1,1,0,0,0,112,0,1
2,112,80207,25971,2010-01-05,License,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,All necessary control measures shall be used ...,1,1,0,0,0,112,0,1
3,113,78239,29086,2010-01-05,License Re-Inspection,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,NO LOG BOOK PROVIDED WITH NECESSARY DOCUMENTS...,1,1,0,0,0,112,0,1
4,114,80208,30639,2010-01-05,License,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,All necessary control measures shall be used ...,1,1,0,0,0,112,0,1


Create `food-inspections-OLTP.csv` for initial Tableau reporting

In [18]:
oltp_csv = read_query("""
SELECT i.inspection_id, e.*, ri.*, i.inspection_date, i.inspection_type, re.*,v.*,iv.comment
FROM inspection i
    LEFT JOIN establishment e on i.establishment_id = e.establishment_id
    LEFT JOIN risk ri on i.risk_id = ri.risk_id
    LEFT JOIN result re on i.result_id = re.result_id
    LEFT JOIN inspection_violation iv on i.inspection_id = iv.inspection_id
    INNER JOIN violation v on iv.violation_id = v.violation_id
""")

oltp_csv.to_csv("food-inspections-OLTP.csv")

Create `food-inspections-OLAP.csv` for final Tableau reporting

In [19]:
olap_csv = read_query("""
SELECT fi.inspection_violation_id,fi.inspection_id,de.*,fi.inspection_date,fi.inspection_type,fi.violation,fi.comment,
    dri.*,fi.high_risk,fi.medium_risk,fi.low_risk,fi.all_risk,dre.*,fi.pass,fi.fail
FROM fact_inspection fi
    INNER JOIN dim_establishment de on fi.establishment_id = de.establishment_id
    INNER JOIN dim_risk dri on fi.risk_id = dri.risk_id
    INNER JOIN dim_result dre on fi.result_id = dre.result_id
""",schema="foodinspectionDW")

olap_csv.to_csv("food-inspections-OLAP.csv")

**Provide metadata metrics**

Below are the list of transformations used to create MySQL DDL and DML scripts

**OLTP**
- Manually associate `risk_id` with each distinct `risk` in the **`risk`** table
- Parsed `result` in the **`result`** table to create an additional `condition_flag` if the inspection had any further check-ups
- Split and stacked `violation` in the **`violation`** and **`inspection_violation`** from one row to many rows as all violations are grouped together parsed by a `|` per `inspection_id`
    - Also split out each `violation` `comment` as each `violation` will have it's own respective `comment`
    - Inspections can have multiple violations and each `violation` will have a `comment` detailing the `violation`
    - This needed to be parsed out in the **`inspection_violation`** table to reduce redundancy in the dataset
- Parsed the `address` components of each `establishment` in the **`establishment`** table using the `usaddress` package
    - This needed to be done in an automated way due to addresses having different formats depending on the area of Chicago
    
**OLAP**
- Create dimension tables for `dim_risk`, `dim_result` and `dim_establishment`
- Created flags that can numerically aggregated based on type of `risk` and `result`

In [20]:
# End script
script_end = datetime.now()
print("Total Script Duration:", script_end - script_start)
print("Seconds for a daily pull")

# Data size
print("\n# of rows:",df.shape[0])
print("# of columns:",df.shape[1])

print("\nOriginal Dataset size (MB):", df.memory_usage().sum()*1e-6)
print("Original Dataset size (GB):", df.memory_usage().sum()*1e-9)

print("\nOLTP dataset size (MB):", oltp_csv.memory_usage().sum()*1e-6)
print("OLTP dataset size (GB):", oltp_csv.memory_usage().sum()*1e-9)

print("\nOLAP dataset size (MB):", olap_csv.memory_usage().sum()*1e-6)
print("OLAP dataset size (GB):", olap_csv.memory_usage().sum()*1e-9)

Total Script Duration: 0:12:23.136886
Seconds for a daily pull

# of rows: 213923
# of columns: 18

Original Dataset size (MB): 30.805039999999998
Original Dataset size (GB): 0.030805040000000002

OLTP dataset size (MB): 129.026448
OLTP dataset size (GB): 0.12902644800000002

OLAP dataset size (MB): 122.36992
OLAP dataset size (GB): 0.12236992000000001
